# Generate music tracks from trained model

In [1]:
import os
import numpy as np
import glob
import pickle
from tqdm import tqdm
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.layers import Activation, Dense, Bidirectional, LSTM, LeakyReLU, BatchNormalization, Reshape, Input
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
PATH_TO_GEN_MODEL = './Models/generator_model.h5'
PATH_TO_RESULT_DIR = './Results/'
latent_dim = 1000

In [3]:
notes_file = open('notes', 'rb')
notes = pickle.load(notes_file)

In [4]:
sequence_length = 100

unique_notes = sorted(set(notes))
unique_notes_count = len(unique_notes)

int_to_note = dict((index, note) for index, note in enumerate(unique_notes))
note_to_int = dict((note, index) for index, note in enumerate(unique_notes))

print('Integer to Note mapping: \n')
print(int_to_note)

Integer to Note mapping: 

{0: '0.3.6', 1: '0.3.7', 2: '0.4.7', 3: '0.4.8', 4: '1.4.7', 5: '1.4.7.9', 6: '1.4.8', 7: '10.1.4.6', 8: '10.2.5', 9: '11.2.4.7', 10: '11.2.5.7', 11: '11.2.6', 12: '11.3.6', 13: '2.5.8', 14: '2.5.8.10', 15: '2.5.9', 16: '2.6.9', 17: '3.6.9.11', 18: '3.7.10', 19: '4.7.10', 20: '4.7.10.0', 21: '4.7.11', 22: '4.8.11', 23: '5.8.11', 24: '5.9.0', 25: '6.10.1', 26: '6.9.0.2', 27: '6.9.0.2.3', 28: '6.9.1', 29: '6.9.11', 30: '6.9.11.2', 31: '7.10.1', 32: '7.10.2', 33: '7.11.2', 34: '7.9.1', 35: '8.11.2.4', 36: '8.11.2.4.5', 37: '9.0.3', 38: '9.0.3.5', 39: '9.0.4', 40: '9.1', 41: '9.1.4', 42: '9.11.2.5', 43: 'A3', 44: 'A4', 45: 'A5', 46: 'B-3', 47: 'B-4', 48: 'B-5', 49: 'B3', 50: 'B4', 51: 'B5', 52: 'C#4', 53: 'C#5', 54: 'C#6', 55: 'C4', 56: 'C5', 57: 'C6', 58: 'D4', 59: 'D5', 60: 'D6', 61: 'E-4', 62: 'E-5', 63: 'E4', 64: 'E5', 65: 'F#4', 66: 'F#5', 67: 'F4', 68: 'F5', 69: 'G#3', 70: 'G#4', 71: 'G#5', 72: 'G3', 73: 'G4', 74: 'G5'}


In [5]:
generator_model = load_model(PATH_TO_GEN_MODEL, compile = False)

In [12]:
noise = np.random.normal(0, 1, (1, latent_dim))
prediction = generator_model.predict(noise)
predicted_notes = [x*(float(unique_notes_count)/2) +(float(unique_notes_count)/2) for x in prediction[0]]
predicted_notes = [int_to_note[int(x)] for x in predicted_notes]
print(predicted_notes)

['F4', 'G5', '0.3.6', 'G5', '0.3.7', '8.11.2.4', '7.11.2', '0.4.7', 'G#5', '10.2.5', 'E4', '11.2.6', 'E5', 'C6', 'F#4', '6.9.0.2', 'G3', 'C4', '0.3.7', '0.4.7', 'B4', '0.4.8', '0.4.8', '8.11.2.4', '0.3.6', '2.5.8', 'C#4', '6.9.11.2', 'C4', '11.2.4.7', '0.3.6', '9.1', '0.3.6', 'G5', '0.3.6', '0.3.6', 'F#4', '0.3.6', '0.3.6', 'C5', '0.3.6', '0.3.6', 'G3', '0.3.6', 'C4', '0.3.6', '11.3.6', 'G5', '0.3.6', '5.8.11', '11.2.4.7', '0.3.6', 'B-5', '9.0.3', '2.5.9', '10.1.4.6', '10.2.5', 'E-4', 'C#5', '10.2.5', '1.4.7.9', 'G5', '10.1.4.6', '6.9.0.2', '4.7.11', '4.7.10.0', 'G3', '0.4.8', '0.3.7', '8.11.2.4', 'D6', 'A4', '0.4.8', '11.2.4.7', 'G4', '2.5.8', '6.9.1', '6.9.11.2', '0.3.7', 'F5', 'C4', 'C4', '1.4.7.9', 'C4', 'G4', '0.3.6', 'G3', 'G#3', 'F#4', 'C#6', 'B3', 'C4', 'G#4', 'E-5', '9.0.4', 'G4', '2.5.9', 'G5', '1.4.8', 'G5']


In [13]:
time_offset = 0
output_notes = []

for element in predicted_notes:
    item = element[0]
    
    # if item is a chord, i.e, if the item is a digit or it contains a dot (.)
    if item.isdigit() or ('.' in item):
        notes_in_chord = item.split('.')
        extracted_notes = []
        
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            extracted_notes.append(new_note)
            
        new_chord = chord.Chord(extracted_notes)
        new_chord.offset = time_offset
        output_notes.append(new_chord)
    else:
        # pattern is a note
        new_note = note.Note(item)
        new_note.offset = time_offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    time_offset += 0.5

new_midi_stream = stream.Stream(output_notes)
new_midi_stream.write('midi', fp=PATH_TO_RESULT_DIR + 'result_GAN.mid')

'./Results/result_GAN.mid'